<a href="https://colab.research.google.com/github/AmirJlr/Thesis/blob/master/notebooks/molfeat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://github.com/datamol-io/molfeat/raw/main/docs/images/logo-title.svg" />

In [2]:

!pip install rdkit
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html
!pip install torch_geometric
!pip install deepchem
!pip install torchinfo
!pip install molfeat

Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.1/947.1 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.1 MB/s eta 0:00:00


In [3]:
import torch
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem

from torch_geometric.data import Dataset, InMemoryDataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_smiles
from torch_geometric.utils import degree

import os
from tqdm.notebook import tqdm
from tqdm.auto import tqdm

import deepchem as dc

from sklearn.model_selection import train_test_split

from molfeat.calc import FPCalculator, MordredDescriptors, Pharmacophore2D, Pharmacophore3D, RDKitDescriptors3D
import datamol as dm
from molfeat.trans import MoleculeTransformer

from sklearn.decomposition import PCA

In [1]:
df = pd.read_csv('/content/basic/raw/bace.csv')
data = df.mol.values

### prepare FP, DES :
calc_ecfp = FPCalculator("ecfp") # on smiles
calc_atompair = FPCalculator("atompair") # on smiles
calc_maccs = FPCalculator("maccs") # on smiles
calc_estate = FPCalculator("estate") # on smiles
calc_mordredD = MordredDescriptors() # on smiles
calc_phar2D = Pharmacophore2D() # on smiles
calc_phar3D = Pharmacophore3D() # on conformer
calc_rdkit3D = RDKitDescriptors3D() # on conformer

featurizer_ecfp = MoleculeTransformer(calc_ecfp, dtype=np.float64)
featurizer_atompair= MoleculeTransformer(calc_atompair, dtype=np.float64)
featurizer_maccs = MoleculeTransformer(calc_maccs, dtype=np.float64)
featurizer_estate = MoleculeTransformer(calc_estate, dtype=np.float64)
featurizer_mordredD = MoleculeTransformer(calc_mordredD, dtype=np.float64)
featurizer_phar2D = MoleculeTransformer(calc_phar2D, dtype=np.float64)
featurizer_phar3D = MoleculeTransformer(calc_phar3D, dtype=np.float64)
featurizer_rdkit3D = MoleculeTransformer(calc_rdkit3D, dtype=np.float64)

def generate_conformers(mol):
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol)  # Generate a conformer
    AllChem.MMFFOptimizeMolecule(mol)  # Optimize the conformer
    return mol

# Assuming 'data' is a list of SMILES strings
mols_with_conformers = [Chem.MolFromSmiles(smiles) for smiles in data]
mols_with_conformers = [generate_conformers(mol) for mol in mols_with_conformers]

with dm.without_rdkit_log():
    ECFP = featurizer_ecfp(data)
    AtomPair = featurizer_atompair(data)
    MACCS = featurizer_maccs(data)
    EState = featurizer_estate(data)
    MordredD = featurizer_mordredD(data)
    Phar2D = featurizer_phar2D(data)
    Phar3D = featurizer_phar3D(mols_with_conformers)
    Rdkit3D = featurizer_rdkit3D(mols_with_conformers)

ModuleNotFoundError: No module named 'molfeat'

In [36]:
N_COMPONENTS = 16

pca_ecfp = PCA(n_components= N_COMPONENTS)
pca_atompair = PCA(n_components= N_COMPONENTS)
pca_maccs = PCA(n_components= N_COMPONENTS)
pca_estate = PCA(n_components= N_COMPONENTS)
pca_mordredD = PCA(n_components= N_COMPONENTS)
pca_phar2D = PCA(n_components= N_COMPONENTS)
pca_phar3D = PCA(n_components= N_COMPONENTS)
pca_rdkit3D = PCA(n_components= N_COMPONENTS)

ECFP_reduced = pca_ecfp.fit_transform(ECFP)
AtomPair_reduced = pca_atompair.fit_transform(AtomPair)
MACCS_reduced = pca_maccs.fit_transform(MACCS)
EState_reduced = pca_estate.fit_transform(EState)
MordredD_reduced = pca_mordredD.fit_transform(MordredD)
Phar2D_reduced = pca_phar2D.fit_transform(Phar2D)
Phar3D_reduced = pca_phar3D.fit_transform(Phar3D)
Rdkit3D_reduced = pca_rdkit3D.fit_transform(Rdkit3D)

assert ECFP_reduced.shape == AtomPair_reduced.shape == MACCS_reduced.shape == EState_reduced.shape == MordredD_reduced.shape == Phar2D_reduced.shape == Phar3D_reduced.shape == Rdkit3D_reduced.shape

(1513, 216)

In [ ]:
class DTsetBasic(InMemoryDataset):
    def __init__(self, root, filename, smiles_column, label_column, rdkit2D):
        self.filename = filename
        self.smiles_column = smiles_column
        self.label_column = label_column

        self.rdkit2D = rdkit2D

        super().__init__(root)
        self.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [self.filename]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        pass  # Implement download logic if needed

    def process(self):
        # Load raw data
        data_path = os.path.join(self.raw_dir, self.filename)
        df = pd.read_csv(data_path)

        graph_list = []
        for i, smiles in tqdm(enumerate(df[self.smiles_column])):

            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                continue  # Skip invalid SMILES strings

            g = from_smiles(smiles)
            g.x = g.x.float()
            y = torch.tensor(df[self.label_column][i], dtype=torch.float).view(1, -1)
            g.y = y

            g.rdkit2D = torch.tensor(self.rdkit2D[i], dtype=torch.float).view(1, -1)

            graph_list.append(g)

        data_list = graph_list

        # Apply pre-filter and pre-transform
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        # Save processed data
        self.save(data_list, self.processed_paths[0])


In [ ]:
dataset = DTsetBasic(root='new', filename='bace.csv', smiles_column='mol', label_column='Class', rdkit2D=rdkit2D_reduced)

## All featurizers in Molfeat inherit from at least one of three classes:

1. `molfeat.calc.SerializableCalculator` : A calculator is a Callable that featurizes a single molecule.

2. `molfeat.trans.MoleculeTransformer` : A transformer is a class that wraps a calculator in a featurization pipeline.

3. `molfeat.trans.pretrained.PretrainedMolTransformer` : A subclass of MoleculeTransformer that extends the transformer interface to support the usage of pretrained models.

## Fingerprints Calculators :
A calculator is a Callable that takes an RDKit Chem.Mol object or a SMILES string and returns a feature vector. In the following example, we will use the FPCalculator.

In [ ]:
# The FPCalculator implements several popular molecular fingerprints:

from molfeat.calc import FP_FUNCS
FP_FUNCS.keys()

dict_keys(['maccs', 'avalon', 'ecfp', 'fcfp', 'topological', 'atompair', 'rdkit', 'pattern', 'layered', 'map4', 'secfp', 'erg', 'estate', 'avalon-count', 'rdkit-count', 'ecfp-count', 'fcfp-count', 'topological-count', 'atompair-count'])

In [ ]:
from molfeat.calc import FPCalculator

smiles = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"
calc = FPCalculator("estate")
X = calc(smiles)
X.shape

(79,)

In [ ]:
# Switching to any other fingerprint is easy:
calc = FPCalculator("ecfp")
X = calc(smiles)
X.shape

(2048,)

## Descriptor Calculator

Beyond these fingerprints, Molfeat also provides **calculators for other molecular descriptors**.

All available calculator classes, both built-in and plugin-based, can be found through the `molfeat.calc` module:

In [ ]:
from molfeat.calc import _CALCULATORS
_CALCULATORS.keys()

dict_keys(['CATS', 'RDKitDescriptors2D', 'RDKitDescriptors3D', 'MordredDescriptors', 'FPCalculator', 'Pharmacophore2D', 'Pharmacophore3D', 'USRDescriptors', 'ElectroShapeDescriptors', 'ScaffoldKeyCalculator', 'AtomCalculator', 'AtomMaterialCalculator', 'DGLCanonicalAtomCalculator', 'DGLWeaveAtomCalculator', 'BondCalculator', 'EdgeMatCalculator', 'DGLCanonicalBondCalculator', 'DGLWeaveEdgeCalculator'])

In [ ]:
from molfeat.calc import RDKitDescriptors2D

smiles = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"
calc = RDKitDescriptors2D()
X = calc(smiles)
X.shape

[14:56:21] Initializing MetalDisconnector
[14:56:21] Running MetalDisconnector
[14:56:21] Initializing Normalizer
[14:56:21] Running Normalizer
[14:56:21] Initializing MetalDisconnector
[14:56:21] Running MetalDisconnector
[14:56:21] Initializing Normalizer
[14:56:21] Running Normalizer
[14:56:21] DEPRECATION WARNING: please use MorganGenerator
[14:56:21] DEPRECATION WARNING: please use MorganGenerator
[14:56:21] DEPRECATION WARNING: please use MorganGenerator


(216,)

In [ ]:
from molfeat.calc import RDKitDescriptors3D, Pharmacophore3D, USRDescriptors

def generate_conformers(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol)  # Generate a conformer
    AllChem.MMFFOptimizeMolecule(mol)  # Optimize the conformer
    return mol

smiles = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"
mols_with_conformers = generate_conformers(smiles)
calc = USRDescriptors()

calc(mols_with_conformers).shape

(12,)

Every calculator is serializable, meaning it can be efficiently stored to — and loaded from — disk. To learn more, please see the tutorial on [saving and loading featurizers](https://molfeat-docs.datamol.io/stable/tutorials/save_and_load.html).

## Transformers

In practice, you won't want to featurize a single molecule, but rather a batch of molecules. This is where transformers come in.

**A transformer is a class that wraps a calculator in a featurization pipeline**.

The MoleculeTransformer class provides a convenient interface for **featurizing a batch of molecules**. It also provides a number of useful methods to customize the featurization pipeline.

In [ ]:
from molfeat.calc import RDKitDescriptors3D
from molfeat.trans import MoleculeTransformer

data = dm.data.freesolv().smiles.values

len(data)

642

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

def generate_conformers(mol):
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol)  # Generate a conformer
    AllChem.MMFFOptimizeMolecule(mol)  # Optimize the conformer
    return mol

# Assuming 'data' is a list of SMILES strings
mols_with_conformers = [Chem.MolFromSmiles(smiles) for smiles in data]
mols_with_conformers = [generate_conformers(mol) for mol in mols_with_conformers]

In [ ]:
# Let's try a different calculator!
calc = RDKitDescriptors3D(replace_nan=True)

featurizer = MoleculeTransformer(calc, n_jobs=1, dtype=torch.float32)

with dm.without_rdkit_log():
    X = featurizer(mols_with_conformers)


In [ ]:
X.shape

torch.Size([642, 639])

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Assuming X is a PyTorch tensor, convert it to a NumPy array
X_np = X.numpy()

# Initialize PCA. You can specify the number of components to keep
pca = PCA(n_components=10)

# Fit and transform the data
X_reduced = pca.fit_transform(X_np)

In [ ]:
X_reduced.shape

(642, 10)

In [ ]:
cumsum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(cumsum>=0.95)+1
print(d)

2


In [ ]:
### RDKit 3D Descriptors
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,629,630,631,632,633,634,635,636,637,638
0,0.605125,0.987798,0.004101,0.155740,0.923234,225.116074,1334.498901,1445.461182,2.895481,1.675609e-01,...,0.307,0.487,0.391,40.571999,20.518999,25.318001,39.636002,28.273001,42.935001,34.752998
1,0.108638,0.821560,0.008930,0.570122,0.980095,109.750389,188.671692,192.503479,1.463826,6.394649e-01,...,0.398,0.638,0.931,5.881000,6.263000,5.486000,6.382000,4.677000,6.238000,8.932000
2,0.271671,0.931039,0.011586,0.364919,0.775449,66.927849,142.220993,183.404739,1.672889,4.287855e-01,...,0.275,0.451,0.332,11.818000,5.015000,7.243000,11.600000,8.355000,12.490000,9.786000
3,0.397587,0.962961,0.007765,0.269639,0.829161,106.776176,328.344818,395.996521,1.960266,2.444328e-01,...,0.299,0.477,0.330,15.533000,7.782000,9.900000,15.062000,11.183000,16.319000,12.263000
4,0.756375,0.996001,0.012073,0.089346,0.953983,79.019684,843.726868,884.425720,2.788523,1.907480e-01,...,0.332,0.500,0.447,24.122000,13.658000,16.549999,24.032000,18.058001,25.240999,23.617001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,0.755546,0.995951,0.005946,0.089896,0.963891,162.111420,1738.211670,1803.327881,3.420901,1.453280e-01,...,0.367,0.544,0.441,43.842999,26.549999,31.146000,43.464001,33.743999,45.733002,43.125000
638,0.165101,0.826996,0.007906,0.562207,0.571108,72.240784,73.384537,128.494949,1.388195,4.042432e-01,...,0.358,0.609,0.437,7.769000,3.112000,4.650000,7.545000,5.466000,8.137000,6.445000
639,0.583493,0.986027,0.008298,0.166587,0.833413,100.435768,502.466919,602.902710,2.221902,7.588825e-12,...,0.239,0.468,0.484,13.035000,8.370000,8.696000,13.239000,8.933000,13.996000,13.196000
640,0.706997,0.993978,0.007029,0.109582,0.923499,131.384323,1107.234985,1198.956543,3.008575,2.046638e-01,...,0.373,0.473,0.482,24.205000,21.320999,19.400000,24.743000,21.311001,25.228001,27.371000


## Concatenate featurizers

In [ ]:
from molfeat.trans.fp import FPVecTransformer

# We will use the FPVecTransformer to automatically create a calculator by name
maccs = FPVecTransformer("maccs", dtype=np.float32)
ecfp4 = FPVecTransformer("ecfp:4", dtype=np.float32)

maccs([smiles]).shape, ecfp4([smiles]).shape

((1, 167), (1, 2000))

## Pretrained transformers

Finally, the `PretrainedMolTransformer` class extends the transformer interface to support the usage of pretrained models.

 This class is a subclass of MoleculeTransformer and inherits all its methods. In addition, it adds the `_embed()`, and `_convert()`.

- `_embed()`: since pre-trained models benefit from batched featurization, this method is called by the transformer instead of the calculator.

- `_convert()`: this method is called by the transformer to convert the input. For example:

- - For a pre-trained language model, we convert from a SMILES string or Mol object to a SELFIES string.

- - For a pre-trained GNN, we convert from a SMILES string or Mol object to a DGL graph.